In [2]:
import os
import pandas as pd
import soundfile as sf
from fnmatch import fnmatch

In [27]:
def make_csv(root,pattern):
    csv_df = []
    ids = os.listdir(root)
    for idx in ids:
        speaker_root = os.path.join(root, idx)
        for path, subdirs, files in os.walk(speaker_root):
            for name in files:
                fullpath = os.path.join(path, name)
                if fnmatch(name, pattern):
                    csv_df.append(
                        {
                            'id':fullpath.split('.wav')[0].split('/')[-3]+'-'+fullpath.split('.wav')[0].split('/')[-2]+'-'+fullpath.split('.wav')[0].split('/')[-1],
                            'speaker_id':idx,
                            'path':fullpath,
                            'audio':fullpath,
                            'label':int(idx.split('id')[1])-10000
                        }
                    )
    return pd.DataFrame(csv_df)

In [28]:
train_root = '/media/oem/LENOVO_USB_HDD/6254_Project/data/voxceleb/dev/wav/'
test_root = '/media/oem/LENOVO_USB_HDD/6254_Project/data/voxceleb/test/wav/'
pattern = "*.wav"
train_df = make_csv(train_root,pattern) 
train_df.to_csv('verification/train.csv',sep=',',index=False)
test_df = make_csv(test_root,pattern)
test_df.to_csv('verification/test.csv',sep=',',index=False)

In [81]:
import pandas as pd
train_df = pd.read_csv('verification/train.csv',sep=',')
test_df = pd.read_csv('verification/test.csv',sep=',')

In [44]:
metadata_df = pd.concat((train_df,test_df))
metadata_df.to_csv('identification/metadata.csv',sep=',',index=False)

In [71]:
splits = pd.read_csv('splits.csv',sep=' ',header=None)
splits.columns = ['split','path']
splits['id'] = splits['path'].str.rstrip('.wav').str.replace('/', '-')
splits = splits.drop(columns = 'path')
splits.to_csv('identification/splits.csv',sep=',',index=False)

In [74]:
md_splits = pd.merge(metadata_df,splits, on='id', how='outer')
md_splits.to_csv("identification/metadata_with_split.csv",sep=',',index=False)

In [87]:
md = pd.read_csv("identification/metadata_with_split.csv",sep=',')

In [91]:
train = md.loc[md['split'] == 1]
train = train.drop(columns = 'split')
dev = md.loc[md['split'] == 2]
dev = dev.drop(columns = 'split')
test = md.loc[md['split'] == 3]
test = test.drop(columns = 'split')
train.to_csv('identification/train.csv',sep=',',index=None)
dev.to_csv('identification/dev.csv',sep=',',index=None)
test.to_csv('identification/test.csv',sep=',',index=None)